In [1]:
#Importing library

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import datetime

from scipy import stats
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#Importing dataset
train = pd.read_csv("dataset/train.csv")
val = pd.read_csv("dataset/train_val1.csv")

We normalize the data so that all values are between 0 and 1. So we first remove 'Date' which is the only non numerical attribute. The we apply range normalisation and at the end we add 'Date'. 

In [3]:
train= (train - train.min()) / (train.max() - train.min())
val= (val - val.min()) / (val.max() - val.min())

We prepare the target valuesy for all the sets

In [4]:
y=train['NumberOfSales']
y_val1=val['NumberOfSales']

We prepare the training input samples for all the sets

In [5]:
X=train.drop(['NumberOfSales'], axis=1)
X_val1=val.drop(['NumberOfSales'], axis=1)

In [ ]:
nt=201
na=12
depth=21

feature_names = list(X.columns)

forest = RandomForestRegressor(max_depth=depth, random_state=0, n_estimators=nt, max_features=na, n_jobs=-1)
forest.fit(X, y)
y_pred=forest.predict(X_val1)
mse_val1=mean_squared_error(y_val1,y_pred)
mae_val1=mean_absolute_error(y_val1,y_pred)
print("con tutte le feature", sqrt(mse_val1), mae_val1)

for feature in feature_names[21:]:
    X=train.drop(['NumberOfSales'], axis=1)
    X_val1=val.drop(['NumberOfSales'], axis=1)
    X=X.drop([feature], axis=1)
    X_val1=X_val1.drop([feature], axis=1)
    forest = RandomForestRegressor(max_depth=depth, random_state=0, n_estimators=nt, max_features=na, n_jobs=-1)
    forest.fit(X, y)
    y_pred=forest.predict(X_val1)
    mse_val1=mean_squared_error(y_val1,y_pred)
    mae_val1=mean_absolute_error(y_val1,y_pred)
    print(feature, sqrt(mse_val1), mae_val1)

con tutte le feature 0.056355195206353854 0.0369289847768
Mean_VisibilityKm 0.06043534070929066 0.039439171994
Mean_Wind_SpeedKm_h 0.05671787148050311 0.037892629004
Min_Dew_PointC 0.05761372791491008 0.0379119881089
